In [ ]:
import uproot
import json
import numpy as np
from vis_functions import *
from rate_estim import *
from tqdm import tqdm

notebook_type = "rates"
params_file = open("/home/juno/arsgav/params.json")
PARAMS = json.load(params_file)
PARAMS['type'] = notebook_type

neptune_run = False
if neptune_run:
    run = neptune.init_run(
        project="legnaro/LegnaroSetup",
        api_token=PARAMS['api_token'],
        tags=[
            PARAMS['path'],
            PARAMS['run_numbers'],
            PARAMS['source_names'],
            PARAMS['output_file'],
            notebook_type,
        ],
        name="LNL",
    )
    
    del PARAMS['api_token']    
    run['PARAMS'] = PARAMS

In [ ]:
runs_list = ["run9 - Bkg, 10', n_th 5",
             "run10 - Co60, 10', n_th 5",
             "run11 - Co60, 10', n_th 5"]

In [ ]:
run_numbers = PARAMS['run_numbers'].split(', ')
path = PARAMS['path']
output_file = PARAMS['output_file']
graph_height = int(PARAMS['graph_height'])
graph_width = int(PARAMS['graph_width'])
left_shift = int(PARAMS['left_shift'])

In [ ]:
rates_array = []
fit_params_array = []
distrs_array = []
for channel_number in tqdm(range(48)):
    try:
        rate_estim_outputs = np.array(
            [rate_estim(path, run_number, output_file, channel_number, Nmeds=10, left_shift=left_shift) for run_number in run_numbers],
            dtype=object
        )

        rates_array.append(rate_estim_outputs[:, 0])
        fit_params_array.append(rate_estim_outputs[:, 1])
        distrs_array.append(rate_estim_outputs[:, 2])
    except:
        pass

## Rates visualization: 

In [ ]:
if neptune_run:
    rates_fit_plots(run_numbers, runs_list, distrs_array, fit_params_array, rates_array,
                    Nbins=50, line_width=0.5,  height=graph_height, width=graph_width,
                    neptune_run=neptune_run, run=run,
                    run_plot_name=f"Rate by channel for {PARAMS['run_numbers']} runs")
else:
    rates_fit_plots(run_numbers, runs_list, distrs_array, fit_params_array, rates_array,
                    Nbins=50, line_width=0.5, height=graph_height, width=graph_width,)

In [ ]:
if neptune_run:
    run.stop()